In [36]:
import requests
import osmnx as ox
import json
import geopandas as gpd
from shapely.geometry import mapping
import os

APIKEY = "my key"
ZONE = "Paris, France"
ROUNDABOUTS = None
DENSITE_POINTS = 8.0
RAYON_SIMULE = 16.0

In [37]:
ROUNDABOUTS = ox.features_from_place(ZONE, tags={'junction': 'roundabout'})
ROUNDABOUTS = ROUNDABOUTS.to_crs(epsg=2154)
ROUNDABOUTS['center'] = ROUNDABOUTS.centroid

ROUNDABOUTS = ROUNDABOUTS.reset_index(drop=True)
ROUNDABOUTS['id'] = (ROUNDABOUTS.index + 1).astype(int)

ROUNDABOUTS = ROUNDABOUTS[['id', 'center', 'geometry']]

masque_points = ROUNDABOUTS.geometry.type == 'Point'
nb_points = masque_points.sum()


if nb_points > 0:
    ROUNDABOUTS.loc[masque_points, 'geometry'] = ROUNDABOUTS.loc[masque_points, 'geometry'] .buffer(RAYON_SIMULE, resolution=4).boundary

ROUNDABOUTS['geometry'] = ROUNDABOUTS['geometry'].segmentize(max_segment_length=DENSITE_POINTS)

ROUNDABOUTS = ROUNDABOUTS.to_crs(epsg=4326)

features = []
for _, row in ROUNDABOUTS.iterrows():
    features.append({
        "type": "Feature",
        "id": int(row['id']),
        "properties": {
            "id": int(row['id']),
            "center": [row['center'].x, row['center'].y]
        },
        "geometry": mapping(row['geometry'])
    })

geojson = {"type": "FeatureCollection", "features": features}

with open("roundabouts.json", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print(f"Nombre de ronds-points trouvés : {len(ROUNDABOUTS)}")


Nombre de ronds-points trouvés : 7


In [38]:
def recup_image_google_street_view(lat, lon, heading=0, fov=90, pitch=0, size="600x400", api_key=APIKEY):
    return f"https://maps.googleapis.com/maps/api/streetview?size={size}&location={lat},{lon}&fov={fov}&heading={heading}&pitch={pitch}&key={api_key}"

In [39]:
def recup_from_coord(lat, long, folder_name, id):
    image_path = os.path.join(folder_name, f"streetview_{id}.jpg")
    url = recup_image_google_street_view(lat, long)
    response = requests.get(url)
    if response.status_code == 200:
        with open(image_path, "wb") as img_file:
            img_file.write(response.content)
    else:
        print(f"Erreur lors du téléchargement pour le rond-point {id}")

In [ ]:
for elem in geojson["features"]:
    id = elem["id"]
    geometry = elem["geometry"]
    folder_name = f"roundabout_{id}"
    os.makedirs(folder_name, exist_ok=True)

    if geometry['type'] == 'Polygon':
        polygone_coords = geometry['coordinates'][0]
    else:
        polygone_coords = geometry['coordinates']
        
    for i, coord in enumerate(polygone_coords):
        lat = coord[1]
        long = coord[0]
        recup_from_coord(lat, long, folder_name, i)